In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

In [2]:
import hopsworks

# Login to the Hopsworks project using your API key
project = hopsworks.login(
    api_key_value="uVSWPn5SRhttOERh.NysMaEwnuCSaxzsMg3MJf9uzIuqM6Jwt8go5sR6hY1pG04ZrW8aM1BwmyywdIcQX")

# Get the feature store instance
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1043609
2024-10-01 16:37:44,145 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-10-01 16:37:44,145 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
feature_group_city_weather = fs.get_feature_group(
    name="city_weather_features", version=1)
feature_group_trucks = fs.get_feature_group(name="trucks_features", version=1)
feature_group_truck_schedule = fs.get_feature_group(
    name="trucks_schedule_features", version=1)
feature_group_routes_weather = fs.get_feature_group(
    name="routes_weather_features", version=1)
feature_group_routes = fs.get_feature_group(name="routes_features", version=1)
feature_group_traffic = fs.get_feature_group(
    name="traffic_features", version=1)
feature_group_drivers = fs.get_feature_group(
    name="drivers_features", version=1)

In [4]:
df_trucks = feature_group_trucks.read()
df_truck_schedule = feature_group_truck_schedule.read()
df_traffic = feature_group_traffic.read()
df_routes_weather = feature_group_routes_weather.read()
df_routes = feature_group_routes.read()
df_drivers = feature_group_drivers.read()
df_city_weather = feature_group_city_weather.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.56s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.76s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.11s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.01s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.52s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.45s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.45s) 


In [5]:
df_routes.shape

(1810, 7)

In [6]:
df_routes_weather.shape

(396149, 11)

In [7]:
df_truck_schedule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   index              12308 non-null  int32              
 1   truck_id           12308 non-null  int32              
 2   route_id           12308 non-null  object             
 3   departure_date     12308 non-null  object             
 4   estimated_arrival  12308 non-null  datetime64[us, UTC]
 5   delay              12308 non-null  int32              
 6   event_date         12308 non-null  datetime64[us, UTC]
dtypes: datetime64[us, UTC](2), int32(3), object(2)
memory usage: 529.0+ KB


In [8]:
df_truck_schedule

,index,truck_id,route_id,departure_date,estimated_arrival,delay,event_date
0,9644,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48+00:00,1,2024-09-28 00:00:00+00:00
1,8231,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24+00:00,0,2024-09-28 00:00:00+00:00
2,2963,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12+00:00,0,2024-09-28 00:00:00+00:00
3,968,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00+00:00,0,2024-09-28 00:00:00+00:00
4,5140,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12+00:00,1,2024-09-28 00:00:00+00:00
...,...,...,...,...,...,...,...
12303,5561,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36+00:00,0,2024-09-28 00:00:00+00:00
12304,4330,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00+00:00,0,2024-09-28 00:00:00+00:00
12305,5290,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36+00:00,0,2024-09-28 00:00:00+00:00
12306,3004,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00+00:00,0,2024-09-28 00:00:00+00:00


In [9]:
from datetime import datetime
import pandas as pd
df_city_weather['datetime'] = df_city_weather['datetime'].astype(str)

# Now you can use the .str accessor
df_city_weather['datetime'] = df_city_weather['datetime'].str.split('+').str[0]

# Optionally convert back to datetime
df_city_weather['datetime'] = pd.to_datetime(df_city_weather['datetime'])

In [10]:
# Step 1: Remove the part after the '+' (including any fractional seconds)
df_truck_schedule['estimated_arrival'] = df_truck_schedule['estimated_arrival'].astype(
    str).str.split('+').str[0].str.split('.').str[0]

# Step 2: Convert to datetime without the fractional seconds
df_truck_schedule['estimated_arrival'] = pd.to_datetime(
    df_truck_schedule['estimated_arrival'], format='%Y-%m-%d %H:%M:%S')

# Display the result
df_truck_schedule

,index,truck_id,route_id,departure_date,estimated_arrival,delay,event_date
0,9644,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2024-09-28 00:00:00+00:00
1,8231,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0,2024-09-28 00:00:00+00:00
2,2963,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2024-09-28 00:00:00+00:00
3,968,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2024-09-28 00:00:00+00:00
4,5140,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2024-09-28 00:00:00+00:00
...,...,...,...,...,...,...,...
12303,5561,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2024-09-28 00:00:00+00:00
12304,4330,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2024-09-28 00:00:00+00:00
12305,5290,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2024-09-28 00:00:00+00:00
12306,3004,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2024-09-28 00:00:00+00:00


In [11]:
df_truck_schedule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   index              12308 non-null  int32              
 1   truck_id           12308 non-null  int32              
 2   route_id           12308 non-null  object             
 3   departure_date     12308 non-null  object             
 4   estimated_arrival  12308 non-null  datetime64[ns]     
 5   delay              12308 non-null  int32              
 6   event_date         12308 non-null  datetime64[us, UTC]
dtypes: datetime64[ns](1), datetime64[us, UTC](1), int32(3), object(2)
memory usage: 529.0+ KB


In [12]:
from datetime import datetime
df_traffic['datetime'] = df_traffic['datetime'].astype(str)

# Now you can use the .str accessor
df_traffic['datetime'] = df_traffic['datetime'].str.split('+').str[0]

# Optionally convert back to datetime
df_traffic['datetime'] = pd.to_datetime(df_traffic['datetime'])

In [13]:
df_trucks = df_trucks.drop(columns=['index', 'event_date'])
df_truck_schedule = df_truck_schedule.drop(columns=['index', 'event_date'])
df_traffic = df_traffic.drop(columns=['index', 'event_date'])
df_routes_weather = df_routes_weather.drop(columns=['index', 'event_date'])
df_routes = df_routes.drop(columns=['index', 'event_date'])
df_drivers = df_drivers.drop(columns=['index', 'event_date'])
df_city_weather = df_city_weather.drop(columns=['index', 'event_date'])

In [14]:
# origin_weather = pd.merge(df_routes, df_city_weather, how='left', left_on='origin_id', right_on='city_id', suffixes=('', '_origin'))

# # Merging for destination city weather
# weather_info = pd.merge(origin_weather, df_city_weather, how='left', left_on='destination_id', right_on='city_id', suffixes=('', '_destination'))

# # Selecting relevant columns

In [15]:
df_truck_schedule

,truck_id,route_id,departure_date,estimated_arrival,delay
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1
...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0


In [16]:
df_routes_weather

,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,R-9604e695,2019-01-09 18:00:00,61,9,Clear,0.0,61,6,1006
1,R-d5cb9f82,2019-01-14 12:00:00,48,3,Partly cloudy,0.0,94,4,1021
2,R-09b4b830,2019-01-16 18:00:00,84,4,Sunny,0.0,21,6,1015
3,R-e56c34fd,2019-01-01 12:00:00,73,7,Sunny,0.0,56,6,1017
4,R-113cebab,2019-01-06 18:00:00,55,1,Sunny,0.0,49,6,1018
...,...,...,...,...,...,...,...,...,...
396144,R-80b8819b,2019-02-02 06:00:00,50,2,Moderate rain,0.1,98,4,1018
396145,R-dd6ff7c1,2019-01-22 00:00:00,77,5,Patchy rain possible,0.0,79,6,1013
396146,R-5b84abef,2019-02-12 18:00:00,36,4,Patchy rain possible,0.0,96,6,1009
396147,R-12cbe9eb,2019-01-06 06:00:00,23,3,Clear,0.0,57,6,1031


In [17]:
df_drivers

,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph
0,be6c4dd8-0,Aaron Kennedy,male,48,8.0,proactive,7,51113858,63.040001
1,3a57cef3-0,Allen Finley,male,45,11.0,conservative,8,13439737,47.509998
2,4ef4816f-d,Amy Beck,female,52,19.0,conservative,4,35547382,40.980000
3,ac199cbb-0,Thomas Harris,male,44,16.0,conservative,4,15561135,39.320000
4,8dc3ec8e-3,Jose Barnes,male,43,7.0,proactive,4,50163414,60.580002
...,...,...,...,...,...,...,...,...,...
1278,615027c8-9,Nathaniel Stevens,None,45,4.0,conservative,7,25681071,50.430000
1279,2cbae81c-7,Joshua Powers,male,46,0.0,proactive,2,11486762,58.340000
1280,fd91f9ba-8,Brian Strickland,male,46,13.0,proactive,5,22469662,61.169998
1281,9f19a100-6,James Kelley,male,56,30.0,proactive,9,18663378,60.290001


In [18]:
df_trucks

,truck_id,truck_age,load_capacity_pounds,mileage_mpg,fuel_type
0,18580086,7,6000.0,24,diesel
1,26225733,10,3000.0,21,diesel
2,22377469,12,15000.0,16,gas
3,96814728,12,3000.0,23,diesel
4,30794834,7,10000.0,18,gas
...,...,...,...,...,...
1196,21865328,10,3000.0,15,gas
1197,27274345,10,15000.0,23,diesel
1198,24624497,11,4000.0,18,gas
1199,12323896,11,15000.0,19,gas


In [19]:
df_city_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-40a81eb1,39,16,Heavy rain,0.2,99,3,1001,2019-01-20 06:00:00
1,C-6df8beaf,28,4,Heavy snow,0.0,95,1,1024,2019-01-12 07:00:00
2,C-639c5e36,27,11,Sunny,0.0,78,6,1022,2019-02-12 16:00:00
3,C-e5bfb4e5,18,6,Light snow,0.0,96,6,1027,2019-02-11 02:00:00
4,C-731988ba,84,6,Sunny,0.0,49,6,1011,2019-02-13 13:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-2aaf0e1a,27,9,Sunny,0.0,31,6,1041,2019-02-09 13:00:00
51830,C-b5282c3b,34,10,Clear,0.0,92,6,1008,2019-01-05 05:00:00
51831,C-c4565ee8,27,4,Cloudy,0.0,92,6,1027,2019-01-17 00:00:00
51832,C-f5ed4c15,18,2,Partly cloudy,0.0,54,6,1025,2019-02-09 19:00:00


In [20]:
df_city_weather.drop_duplicates(subset=['city_id', 'datetime'], inplace=True)
df_routes_weather.drop_duplicates(subset=['route_id', 'date'], inplace=True)
df_trucks.drop_duplicates(subset=['truck_id'], inplace=True)
df_drivers.drop_duplicates(subset=['driver_id'], inplace=True)
df_routes.drop_duplicates(
    subset=['route_id', 'destination_id', 'origin_id'], inplace=True)
df_truck_schedule.drop_duplicates(
    subset=['truck_id', 'route_id', 'departure_date'], inplace=True)

In [21]:
df_city_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-40a81eb1,39,16,Heavy rain,0.2,99,3,1001,2019-01-20 06:00:00
1,C-6df8beaf,28,4,Heavy snow,0.0,95,1,1024,2019-01-12 07:00:00
2,C-639c5e36,27,11,Sunny,0.0,78,6,1022,2019-02-12 16:00:00
3,C-e5bfb4e5,18,6,Light snow,0.0,96,6,1027,2019-02-11 02:00:00
4,C-731988ba,84,6,Sunny,0.0,49,6,1011,2019-02-13 13:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-2aaf0e1a,27,9,Sunny,0.0,31,6,1041,2019-02-09 13:00:00
51830,C-b5282c3b,34,10,Clear,0.0,92,6,1008,2019-01-05 05:00:00
51831,C-c4565ee8,27,4,Cloudy,0.0,92,6,1027,2019-01-17 00:00:00
51832,C-f5ed4c15,18,2,Partly cloudy,0.0,54,6,1025,2019-02-09 19:00:00


In [22]:
df_routes_weather

,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,R-9604e695,2019-01-09 18:00:00,61,9,Clear,0.0,61,6,1006
1,R-d5cb9f82,2019-01-14 12:00:00,48,3,Partly cloudy,0.0,94,4,1021
2,R-09b4b830,2019-01-16 18:00:00,84,4,Sunny,0.0,21,6,1015
3,R-e56c34fd,2019-01-01 12:00:00,73,7,Sunny,0.0,56,6,1017
4,R-113cebab,2019-01-06 18:00:00,55,1,Sunny,0.0,49,6,1018
...,...,...,...,...,...,...,...,...,...
396144,R-80b8819b,2019-02-02 06:00:00,50,2,Moderate rain,0.1,98,4,1018
396145,R-dd6ff7c1,2019-01-22 00:00:00,77,5,Patchy rain possible,0.0,79,6,1013
396146,R-5b84abef,2019-02-12 18:00:00,36,4,Patchy rain possible,0.0,96,6,1009
396147,R-12cbe9eb,2019-01-06 06:00:00,23,3,Clear,0.0,57,6,1031


In [23]:
import pandas as pd

# Assuming df_truck_schedule and df_routes_weather are already loaded and datetime columns are properly formatted
df_truck_schedule['estimated_arrival'] = pd.to_datetime(
    df_truck_schedule['estimated_arrival'])
df_truck_schedule['departure_date'] = pd.to_datetime(
    df_truck_schedule['departure_date'])
df_routes_weather['date'] = pd.to_datetime(df_routes_weather['date'])

# Round the estimated_arrival to the nearest upper 6 hours and departure_date to the nearest lower 6 hours
df_truck_schedule1 = df_truck_schedule.copy()

In [24]:
df_truck_schedule

,truck_id,route_id,departure_date,estimated_arrival,delay
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1
...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0


In [25]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1
...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0


In [26]:
df_truck_schedule1['estimated_arrival'] = df_truck_schedule1['estimated_arrival'].dt.ceil(
    '6H')
df_truck_schedule1['departure_date'] = df_truck_schedule1['departure_date'].dt.floor(
    '6H')

In [27]:
df_truck_schedule1['date'] = [
    pd.date_range(start=row['departure_date'],
                  end=row['estimated_arrival'], freq='6H')
    for _, row in df_truck_schedule1.iterrows()
]

In [28]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,"DatetimeIndex(['2019-01-11 06:00:00', '2019-01..."
1,61984883,R-d87e53cd,2019-02-09 06:00:00,2019-02-10 00:00:00,0,"DatetimeIndex(['2019-02-09 06:00:00', '2019-02..."
2,18663378,R-d5d0786c,2019-01-13 06:00:00,2019-01-13 18:00:00,0,"DatetimeIndex(['2019-01-13 06:00:00', '2019-01..."
3,23496767,R-6717548a,2019-01-04 06:00:00,2019-01-04 12:00:00,0,"DatetimeIndex(['2019-01-04 06:00:00', '2019-01..."
4,32898016,R-c9bd5880,2019-01-25 06:00:00,2019-01-25 18:00:00,1,"DatetimeIndex(['2019-01-25 06:00:00', '2019-01..."
...,...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 06:00:00,2019-01-29 06:00:00,0,"DatetimeIndex(['2019-01-28 06:00:00', '2019-01..."
12304,10489983,R-7738be4f,2019-01-22 06:00:00,2019-01-23 06:00:00,0,"DatetimeIndex(['2019-01-22 06:00:00', '2019-01..."
12305,24034677,R-4febf0c1,2019-01-25 06:00:00,2019-01-26 00:00:00,0,"DatetimeIndex(['2019-01-25 06:00:00', '2019-01..."
12306,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,"DatetimeIndex(['2019-01-13 06:00:00', '2019-01..."


In [29]:
df_truck_schedule1 = df_truck_schedule1.explode('date')

In [30]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 06:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 12:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 18:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-12 00:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-12 06:00:00
...,...,...,...,...,...,...
12306,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,2019-01-13 12:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 06:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 12:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 18:00:00


In [31]:
df_truck_schedule1 = df_truck_schedule1.drop_duplicates()

In [32]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 06:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 12:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 18:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-12 00:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-12 06:00:00
...,...,...,...,...,...,...
12306,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,2019-01-13 12:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 06:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 12:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 18:00:00


In [33]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 06:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 12:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 18:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-12 00:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-12 06:00:00
...,...,...,...,...,...,...
12306,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,2019-01-13 12:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 06:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 12:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 18:00:00


In [34]:


# Now merge the dataframes based on 'route_id' and the rounded dates
# Merging on route_id and matching the departure date with the weather date for that route
df_merged = pd.merge(
    df_truck_schedule1,
    df_routes_weather,
    on=['route_id', 'date'],
    how='left'  # Using left join to retain all records from truck schedule even if no weather data is found
)

# Drop the 'date' column from weather data as it's redundant now
df_merged.drop(columns=['date'], inplace=True)

# The result is a merged dataframe with truck schedules and corresponding route weather
df_merged

,truck_id,route_id,departure_date,estimated_arrival,delay,temp,wind_speed,description,precip,humidity,visibility,pressure
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,4.0,Light rain shower,0.0,98.0,4.0,1013.0
1,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,52.0,4.0,Sunny,0.0,39.0,6.0,1014.0
2,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,55.0,2.0,Sunny,0.0,38.0,6.0,1014.0
3,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,5.0,Fog,0.0,98.0,2.0,1013.0
4,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,6.0,Fog,0.0,98.0,0.0,1012.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71371,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,41.0,11.0,Cloudy,0.0,70.0,6.0,1022.0
71372,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71373,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,68.0,9.0,Patchy rain possible,0.0,88.0,6.0,1017.0
71374,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,70.0,11.0,Patchy rain possible,0.0,88.0,6.0,1017.0


In [35]:
df_merged.isnull().sum()

truck_id                0
route_id                0
departure_date          0
estimated_arrival       0
delay                   0
temp                 5199
wind_speed           5199
description          5199
precip               5199
humidity             5199
visibility           5199
pressure             5199
dtype: int64

In [36]:
df_merged = df_merged.dropna(
    subset=['temp', 'wind_speed', 'description', 'humidity', 'pressure'])
df_merged

,truck_id,route_id,departure_date,estimated_arrival,delay,temp,wind_speed,description,precip,humidity,visibility,pressure
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,4.0,Light rain shower,0.0,98.0,4.0,1013.0
1,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,52.0,4.0,Sunny,0.0,39.0,6.0,1014.0
2,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,55.0,2.0,Sunny,0.0,38.0,6.0,1014.0
3,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,5.0,Fog,0.0,98.0,2.0,1013.0
4,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,6.0,Fog,0.0,98.0,0.0,1012.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71369,24034677,R-4febf0c1,2019-01-25 06:00:00,2019-01-26 00:00:00,0,63.0,6.0,Clear,0.0,28.0,6.0,1021.0
71370,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,79.0,6.0,Partly cloudy,0.0,60.0,6.0,1015.0
71371,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,41.0,11.0,Cloudy,0.0,70.0,6.0,1022.0
71373,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,68.0,9.0,Patchy rain possible,0.0,88.0,6.0,1017.0


In [37]:
df_merged

,truck_id,route_id,departure_date,estimated_arrival,delay,temp,wind_speed,description,precip,humidity,visibility,pressure
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,4.0,Light rain shower,0.0,98.0,4.0,1013.0
1,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,52.0,4.0,Sunny,0.0,39.0,6.0,1014.0
2,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,55.0,2.0,Sunny,0.0,38.0,6.0,1014.0
3,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,5.0,Fog,0.0,98.0,2.0,1013.0
4,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,6.0,Fog,0.0,98.0,0.0,1012.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71369,24034677,R-4febf0c1,2019-01-25 06:00:00,2019-01-26 00:00:00,0,63.0,6.0,Clear,0.0,28.0,6.0,1021.0
71370,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,79.0,6.0,Partly cloudy,0.0,60.0,6.0,1015.0
71371,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,41.0,11.0,Cloudy,0.0,70.0,6.0,1022.0
71373,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,68.0,9.0,Patchy rain possible,0.0,88.0,6.0,1017.0


In [38]:
df_merged.isnull().sum()

truck_id             0
route_id             0
departure_date       0
estimated_arrival    0
delay                0
temp                 0
wind_speed           0
description          0
precip               0
humidity             0
visibility           0
pressure             0
dtype: int64

In [39]:
df_merged.shape

(66177, 12)

In [40]:
def custom_mode(x):
    """Calculate the mode of a pandas Series."""
    return x.mode().iloc[0]

In [41]:
df_merged

,truck_id,route_id,departure_date,estimated_arrival,delay,temp,wind_speed,description,precip,humidity,visibility,pressure
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,4.0,Light rain shower,0.0,98.0,4.0,1013.0
1,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,52.0,4.0,Sunny,0.0,39.0,6.0,1014.0
2,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,55.0,2.0,Sunny,0.0,38.0,6.0,1014.0
3,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,5.0,Fog,0.0,98.0,2.0,1013.0
4,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,68.0,6.0,Fog,0.0,98.0,0.0,1012.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71369,24034677,R-4febf0c1,2019-01-25 06:00:00,2019-01-26 00:00:00,0,63.0,6.0,Clear,0.0,28.0,6.0,1021.0
71370,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,79.0,6.0,Partly cloudy,0.0,60.0,6.0,1015.0
71371,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,41.0,11.0,Cloudy,0.0,70.0,6.0,1022.0
71373,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,68.0,9.0,Patchy rain possible,0.0,88.0,6.0,1017.0


In [42]:
scheduled_weather = pd.DataFrame(df_merged)
schedule_weather_grp = scheduled_weather.groupby(['truck_id', 'route_id'], as_index=False).agg(
    route_avg_temp=('temp', 'mean'),
    route_avg_wind_speed=('wind_speed', 'mean'),

    route_avg_humidity=('humidity', 'mean'),

    route_avg_pressure=('pressure', 'mean'),
    route_avg_precip = ('precip','mean'),
    route_avg_visibility = ('visibility','mean'),
    route_description=('description', custom_mode)
)

# Display the aggregated DataFrame
schedule_weather_grp

,truck_id,route_id,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,10008392,R-06dfe39e,50.750000,6.416667,79.416667,1015.250000,0.00,5.833333,Partly cloudy
1,10008392,R-35dca729,56.600000,9.400000,72.200000,1013.300000,0.00,5.700000,Partly cloudy
2,10008392,R-51d1a32a,46.636364,8.909091,76.181818,1016.818182,0.00,6.000000,Partly cloudy
3,10008392,R-7ab5ae85,58.750000,7.000000,82.250000,1016.000000,0.00,4.000000,Sunny
4,10008392,R-c6f8e036,47.916667,6.166667,88.500000,1011.000000,0.00,5.583333,Clear
...,...,...,...,...,...,...,...,...,...
12192,99981667,R-abbf83d1,59.000000,7.500000,80.750000,1014.500000,0.00,5.500000,Light rain shower
12193,99981667,R-e9bf94ff,58.400000,3.000000,70.800000,1016.400000,0.00,4.400000,Sunny
12194,99981667,R-ed1094fd,64.500000,10.500000,42.500000,1016.750000,0.00,6.000000,Clear
12195,99981667,R-f363ad37,79.000000,11.800000,84.800000,1013.600000,0.02,4.800000,Clear


In [43]:
# schedule_weather_grp = schedule_weather_grp.rename(
#     columns={'index_x': 'index'})

In [44]:
schedule_weather_merge = df_truck_schedule.merge(
    schedule_weather_grp,
    on=['truck_id', 'route_id'],
    how='left'
)

# Display the merged DataFrame
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,61.285714,4.000000,62.142857,1013.285714,0.0,4.285714,Sunny
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0,80.500000,10.000000,63.500000,1011.250000,0.0,4.250000,Patchy light drizzle
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.000000,0.0,6.000000,Clear
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.500000,0.0,6.000000,Clear
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.000000,0.0,4.333333,Moderate snow
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.200000,0.0,6.000000,Sunny
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.000000,0.0,5.000000,Sunny
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,63.000000,6.000000,27.500000,1021.000000,0.0,6.000000,Clear
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,60.000000,8.500000,65.000000,1018.500000,0.0,6.000000,Cloudy


In [45]:
schedule_weather_merge.isnull().sum()

truck_id                 0
route_id                 0
departure_date           0
estimated_arrival        0
delay                    0
route_avg_temp          43
route_avg_wind_speed    43
route_avg_humidity      43
route_avg_pressure      43
route_avg_precip        43
route_avg_visibility    43
route_description       43
dtype: int64

In [46]:
schedule_weather_merge = schedule_weather_merge.dropna()
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,61.285714,4.000000,62.142857,1013.285714,0.0,4.285714,Sunny
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0,80.500000,10.000000,63.500000,1011.250000,0.0,4.250000,Patchy light drizzle
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.000000,0.0,6.000000,Clear
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.500000,0.0,6.000000,Clear
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.000000,0.0,4.333333,Moderate snow
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.200000,0.0,6.000000,Sunny
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.000000,0.0,5.000000,Sunny
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,63.000000,6.000000,27.500000,1021.000000,0.0,6.000000,Clear
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,60.000000,8.500000,65.000000,1018.500000,0.0,6.000000,Cloudy


In [47]:
# schedule_weather_merge = schedule_weather_merge.drop(columns='date')

In [48]:
schedule_weather_merge = schedule_weather_merge.drop_duplicates()
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,61.285714,4.000000,62.142857,1013.285714,0.0,4.285714,Sunny
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0,80.500000,10.000000,63.500000,1011.250000,0.0,4.250000,Patchy light drizzle
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.000000,0.0,6.000000,Clear
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.500000,0.0,6.000000,Clear
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.000000,0.0,4.333333,Moderate snow
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.200000,0.0,6.000000,Sunny
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.000000,0.0,5.000000,Sunny
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,63.000000,6.000000,27.500000,1021.000000,0.0,6.000000,Clear
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,60.000000,8.500000,65.000000,1018.500000,0.0,6.000000,Cloudy


In [49]:
schedule_weather_merge.isnull().any()

truck_id                False
route_id                False
departure_date          False
estimated_arrival       False
delay                   False
route_avg_temp          False
route_avg_wind_speed    False
route_avg_humidity      False
route_avg_pressure      False
route_avg_precip        False
route_avg_visibility    False
route_description       False
dtype: bool

In [50]:
schedule_weather_merge = schedule_weather_merge.drop_duplicates()

In [51]:
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,61.285714,4.000000,62.142857,1013.285714,0.0,4.285714,Sunny
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0,80.500000,10.000000,63.500000,1011.250000,0.0,4.250000,Patchy light drizzle
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.000000,0.0,6.000000,Clear
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.500000,0.0,6.000000,Clear
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.000000,0.0,4.333333,Moderate snow
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.200000,0.0,6.000000,Sunny
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.000000,0.0,5.000000,Sunny
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,63.000000,6.000000,27.500000,1021.000000,0.0,6.000000,Clear
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,60.000000,8.500000,65.000000,1018.500000,0.0,6.000000,Cloudy


In [52]:
df_city_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-40a81eb1,39,16,Heavy rain,0.2,99,3,1001,2019-01-20 06:00:00
1,C-6df8beaf,28,4,Heavy snow,0.0,95,1,1024,2019-01-12 07:00:00
2,C-639c5e36,27,11,Sunny,0.0,78,6,1022,2019-02-12 16:00:00
3,C-e5bfb4e5,18,6,Light snow,0.0,96,6,1027,2019-02-11 02:00:00
4,C-731988ba,84,6,Sunny,0.0,49,6,1011,2019-02-13 13:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-2aaf0e1a,27,9,Sunny,0.0,31,6,1041,2019-02-09 13:00:00
51830,C-b5282c3b,34,10,Clear,0.0,92,6,1008,2019-01-05 05:00:00
51831,C-c4565ee8,27,4,Cloudy,0.0,92,6,1027,2019-01-17 00:00:00
51832,C-f5ed4c15,18,2,Partly cloudy,0.0,54,6,1025,2019-02-09 19:00:00


In [53]:
df_routes_weather

,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,R-9604e695,2019-01-09 18:00:00,61,9,Clear,0.0,61,6,1006
1,R-d5cb9f82,2019-01-14 12:00:00,48,3,Partly cloudy,0.0,94,4,1021
2,R-09b4b830,2019-01-16 18:00:00,84,4,Sunny,0.0,21,6,1015
3,R-e56c34fd,2019-01-01 12:00:00,73,7,Sunny,0.0,56,6,1017
4,R-113cebab,2019-01-06 18:00:00,55,1,Sunny,0.0,49,6,1018
...,...,...,...,...,...,...,...,...,...
396144,R-80b8819b,2019-02-02 06:00:00,50,2,Moderate rain,0.1,98,4,1018
396145,R-dd6ff7c1,2019-01-22 00:00:00,77,5,Patchy rain possible,0.0,79,6,1013
396146,R-5b84abef,2019-02-12 18:00:00,36,4,Patchy rain possible,0.0,96,6,1009
396147,R-12cbe9eb,2019-01-06 06:00:00,23,3,Clear,0.0,57,6,1031


In [54]:
df_routes

,route_id,origin_id,destination_id,distance,average_hours
0,R-bd779f9f,C-01660979,C-639c5e36,1947.060059,38.939999
1,R-af4cb383,C-841ebdcb,C-3dbd8b2e,1865.439941,37.310001
2,R-83244f30,C-e6dcda92,C-56e39a5e,1664.760010,33.299999
3,R-fea9d26f,C-56b68559,C-c92599e2,1962.319946,39.250000
4,R-647d7ccc,C-927ceb5e,C-825b2625,110.400002,2.210000
...,...,...,...,...,...
1805,R-af435402,C-841ebdcb,C-b6e04c88,692.059998,13.840000
1806,R-5b0c8c66,C-56b68559,C-84f378bb,559.020020,11.180000
1807,R-e2d68801,C-90e30162,C-328bd8d3,557.549988,11.150000
1808,R-d1612204,C-927ceb5e,C-b5ff31cd,1014.750000,20.299999


In [55]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 06:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 12:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-11 18:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-12 00:00:00
0,26716985,R-d348d1e2,2019-01-11 06:00:00,2019-01-12 18:00:00,1,2019-01-12 06:00:00
...,...,...,...,...,...,...
12306,33921946,R-24c815d8,2019-01-13 06:00:00,2019-01-13 12:00:00,0,2019-01-13 12:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 06:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 12:00:00
12307,14730346,R-dda44c57,2019-01-16 06:00:00,2019-01-17 00:00:00,0,2019-01-16 18:00:00


In [56]:
nearest_hour_schedule_df = df_truck_schedule.copy()

# Step 2: Round 'estimated_arrival' and 'departure_date' to the nearest hour
nearest_hour_schedule_df['estimated_arrival_nearest_hour'] = nearest_hour_schedule_df['estimated_arrival'].dt.round(
    "H")
nearest_hour_schedule_df['departure_date_nearest_hour'] = nearest_hour_schedule_df['departure_date'].dt.round(
    "H")

# Step 3: Merge with routes DataFrame to get origin and destination
nearest_hour_schedule_route_df = pd.merge(
    nearest_hour_schedule_df,
    df_routes,
    on='route_id',
    how='left'
)

# Display the resulting DataFrame
nearest_hour_schedule_route_df

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,34.380001
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0,2019-02-09 21:00:00,2019-02-09 07:00:00,C-fc66f0ab,C-5d86b887,681.780029,13.640000
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,7.620000
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,4.600000
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,9.870000
...,...,...,...,...,...,...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,20.809999
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,20.750000
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,15.810000
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,2.150000


In [57]:
nearest_hour_schedule_route_df.isnull().sum()

truck_id                            0
route_id                            0
departure_date                      0
estimated_arrival                   0
delay                               0
estimated_arrival_nearest_hour      0
departure_date_nearest_hour         0
origin_id                         859
destination_id                    859
distance                          859
average_hours                     859
dtype: int64

In [58]:
nearest_hour_schedule_route_df = nearest_hour_schedule_route_df.dropna()
nearest_hour_schedule_route_df

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,34.380001
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0,2019-02-09 21:00:00,2019-02-09 07:00:00,C-fc66f0ab,C-5d86b887,681.780029,13.640000
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,7.620000
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,4.600000
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,9.870000
...,...,...,...,...,...,...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,20.809999
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,20.750000
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,15.810000
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,2.150000


In [59]:
nearest_hour_schedule_df.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
dtype: int64

In [60]:
origin_weather = df_city_weather.copy()
destination_weather = df_city_weather.copy()

In [61]:
# origin_weather = origin_weather.drop(columns=['datetime'])

In [62]:
# destination_weather = destination_weather.drop(columns=['datetime'])

In [63]:
origin_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-40a81eb1,39,16,Heavy rain,0.2,99,3,1001,2019-01-20 06:00:00
1,C-6df8beaf,28,4,Heavy snow,0.0,95,1,1024,2019-01-12 07:00:00
2,C-639c5e36,27,11,Sunny,0.0,78,6,1022,2019-02-12 16:00:00
3,C-e5bfb4e5,18,6,Light snow,0.0,96,6,1027,2019-02-11 02:00:00
4,C-731988ba,84,6,Sunny,0.0,49,6,1011,2019-02-13 13:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-2aaf0e1a,27,9,Sunny,0.0,31,6,1041,2019-02-09 13:00:00
51830,C-b5282c3b,34,10,Clear,0.0,92,6,1008,2019-01-05 05:00:00
51831,C-c4565ee8,27,4,Cloudy,0.0,92,6,1027,2019-01-17 00:00:00
51832,C-f5ed4c15,18,2,Partly cloudy,0.0,54,6,1025,2019-02-09 19:00:00


In [64]:
df_routes

,route_id,origin_id,destination_id,distance,average_hours
0,R-bd779f9f,C-01660979,C-639c5e36,1947.060059,38.939999
1,R-af4cb383,C-841ebdcb,C-3dbd8b2e,1865.439941,37.310001
2,R-83244f30,C-e6dcda92,C-56e39a5e,1664.760010,33.299999
3,R-fea9d26f,C-56b68559,C-c92599e2,1962.319946,39.250000
4,R-647d7ccc,C-927ceb5e,C-825b2625,110.400002,2.210000
...,...,...,...,...,...
1805,R-af435402,C-841ebdcb,C-b6e04c88,692.059998,13.840000
1806,R-5b0c8c66,C-56b68559,C-84f378bb,559.020020,11.180000
1807,R-e2d68801,C-90e30162,C-328bd8d3,557.549988,11.150000
1808,R-d1612204,C-927ceb5e,C-b5ff31cd,1014.750000,20.299999


In [65]:
# origin_weather = pd.merge(df_routes, df_city_weather, how='left', left_on='origin_id', right_on='city_id', suffixes=('', '_origin'))

# Merging for destination city weather
# weather_info = pd.merge(origin_weather, df_city_weather, how='left', left_on='destination_id', right_on='city_id', suffixes=('', '_destination'))

# Selecting relevant columns

In [66]:
nearest_hour_schedule_df

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0,2019-02-09 21:00:00,2019-02-09 07:00:00
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00
...,...,...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00


In [67]:
origin_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-40a81eb1,39,16,Heavy rain,0.2,99,3,1001,2019-01-20 06:00:00
1,C-6df8beaf,28,4,Heavy snow,0.0,95,1,1024,2019-01-12 07:00:00
2,C-639c5e36,27,11,Sunny,0.0,78,6,1022,2019-02-12 16:00:00
3,C-e5bfb4e5,18,6,Light snow,0.0,96,6,1027,2019-02-11 02:00:00
4,C-731988ba,84,6,Sunny,0.0,49,6,1011,2019-02-13 13:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-2aaf0e1a,27,9,Sunny,0.0,31,6,1041,2019-02-09 13:00:00
51830,C-b5282c3b,34,10,Clear,0.0,92,6,1008,2019-01-05 05:00:00
51831,C-c4565ee8,27,4,Cloudy,0.0,92,6,1027,2019-01-17 00:00:00
51832,C-f5ed4c15,18,2,Partly cloudy,0.0,54,6,1025,2019-02-09 19:00:00


In [68]:
origin_weather_merge = pd.merge(nearest_hour_schedule_route_df,
                                origin_weather, left_on=['origin_id', 'departure_date_nearest_hour'], right_on=['city_id', 'datetime'], how='left')

In [69]:
origin_weather_merge.shape

(11449, 20)

In [70]:
origin_weather_merge.isnull().sum()

truck_id                            0
route_id                            0
departure_date                      0
estimated_arrival                   0
delay                               0
estimated_arrival_nearest_hour      0
departure_date_nearest_hour         0
origin_id                           0
destination_id                      0
distance                            0
average_hours                       0
city_id                           552
temp                              552
wind_speed                        552
description                       552
precip                            552
humidity                          552
visibility                        552
pressure                          552
datetime                          552
dtype: int64

In [71]:
origin_weather_merge = origin_weather_merge.dropna()

In [72]:
origin_weather_merge.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
city_id                           0
temp                              0
wind_speed                        0
description                       0
precip                            0
humidity                          0
visibility                        0
pressure                          0
datetime                          0
dtype: int64

In [73]:
origin_weather_merge.shape

(10897, 20)

In [74]:
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,34.380001,C-c92599e2,68.0,4.0,Light rain shower,0.0,92.0,6.0,1019.0,2019-01-11 07:00:00
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,7.620000,C-280b55fb,23.0,4.0,Cloudy,0.0,92.0,6.0,1032.0,2019-01-13 07:00:00
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,4.600000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0,2019-01-04 07:00:00
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,9.870000,C-ef47bdcd,16.0,9.0,Sunny,0.0,67.0,6.0,1029.0,2019-01-25 07:00:00
5,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,33.369999,C-ef47bdcd,28.0,7.0,Heavy snow,0.0,85.0,1.0,1017.0,2019-01-11 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,20.809999,C-419cd14c,54.0,5.0,Partly cloudy,0.0,89.0,6.0,1014.0,2019-01-28 07:00:00
11445,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,20.750000,C-419cd14c,63.0,6.0,Overcast,0.0,81.0,6.0,1015.0,2019-01-22 07:00:00
11446,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,15.810000,C-e5bfb4e5,-4.0,6.0,Overcast,0.0,84.0,6.0,1025.0,2019-01-25 07:00:00
11447,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,2.150000,C-6df8beaf,28.0,6.0,Moderate snow,0.0,93.0,2.0,1028.0,2019-01-13 07:00:00


In [75]:
origin_weather_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
       'origin_id', 'destination_id', 'distance', 'average_hours', 'city_id',
       'temp', 'wind_speed', 'description', 'precip', 'humidity', 'visibility',
       'pressure', 'datetime'],
      dtype='object')

In [76]:
origin_weather_merge = origin_weather_merge.dropna()
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,34.380001,C-c92599e2,68.0,4.0,Light rain shower,0.0,92.0,6.0,1019.0,2019-01-11 07:00:00
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,7.620000,C-280b55fb,23.0,4.0,Cloudy,0.0,92.0,6.0,1032.0,2019-01-13 07:00:00
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,4.600000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0,2019-01-04 07:00:00
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,9.870000,C-ef47bdcd,16.0,9.0,Sunny,0.0,67.0,6.0,1029.0,2019-01-25 07:00:00
5,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,33.369999,C-ef47bdcd,28.0,7.0,Heavy snow,0.0,85.0,1.0,1017.0,2019-01-11 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,20.809999,C-419cd14c,54.0,5.0,Partly cloudy,0.0,89.0,6.0,1014.0,2019-01-28 07:00:00
11445,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,20.750000,C-419cd14c,63.0,6.0,Overcast,0.0,81.0,6.0,1015.0,2019-01-22 07:00:00
11446,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,15.810000,C-e5bfb4e5,-4.0,6.0,Overcast,0.0,84.0,6.0,1025.0,2019-01-25 07:00:00
11447,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,2.150000,C-6df8beaf,28.0,6.0,Moderate snow,0.0,93.0,2.0,1028.0,2019-01-13 07:00:00


In [77]:
# origin_weather_merge = origin_weather_merge.drop(columns='date')
origin_weather_merge = origin_weather_merge.drop_duplicates()
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,34.380001,C-c92599e2,68.0,4.0,Light rain shower,0.0,92.0,6.0,1019.0,2019-01-11 07:00:00
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,7.620000,C-280b55fb,23.0,4.0,Cloudy,0.0,92.0,6.0,1032.0,2019-01-13 07:00:00
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,4.600000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0,2019-01-04 07:00:00
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,9.870000,C-ef47bdcd,16.0,9.0,Sunny,0.0,67.0,6.0,1029.0,2019-01-25 07:00:00
5,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,33.369999,C-ef47bdcd,28.0,7.0,Heavy snow,0.0,85.0,1.0,1017.0,2019-01-11 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,20.809999,C-419cd14c,54.0,5.0,Partly cloudy,0.0,89.0,6.0,1014.0,2019-01-28 07:00:00
11445,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,20.750000,C-419cd14c,63.0,6.0,Overcast,0.0,81.0,6.0,1015.0,2019-01-22 07:00:00
11446,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,15.810000,C-e5bfb4e5,-4.0,6.0,Overcast,0.0,84.0,6.0,1025.0,2019-01-25 07:00:00
11447,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,2.150000,C-6df8beaf,28.0,6.0,Moderate snow,0.0,93.0,2.0,1028.0,2019-01-13 07:00:00


In [78]:
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,34.380001,C-c92599e2,68.0,4.0,Light rain shower,0.0,92.0,6.0,1019.0,2019-01-11 07:00:00
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,7.620000,C-280b55fb,23.0,4.0,Cloudy,0.0,92.0,6.0,1032.0,2019-01-13 07:00:00
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,4.600000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0,2019-01-04 07:00:00
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,9.870000,C-ef47bdcd,16.0,9.0,Sunny,0.0,67.0,6.0,1029.0,2019-01-25 07:00:00
5,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,33.369999,C-ef47bdcd,28.0,7.0,Heavy snow,0.0,85.0,1.0,1017.0,2019-01-11 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,20.809999,C-419cd14c,54.0,5.0,Partly cloudy,0.0,89.0,6.0,1014.0,2019-01-28 07:00:00
11445,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,20.750000,C-419cd14c,63.0,6.0,Overcast,0.0,81.0,6.0,1015.0,2019-01-22 07:00:00
11446,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,15.810000,C-e5bfb4e5,-4.0,6.0,Overcast,0.0,84.0,6.0,1025.0,2019-01-25 07:00:00
11447,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,2.150000,C-6df8beaf,28.0,6.0,Moderate snow,0.0,93.0,2.0,1028.0,2019-01-13 07:00:00


In [79]:
origin_weather_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
       'origin_id', 'destination_id', 'distance', 'average_hours', 'city_id',
       'temp', 'wind_speed', 'description', 'precip', 'humidity', 'visibility',
       'pressure', 'datetime'],
      dtype='object')

In [80]:
origin_weather_merge.rename(columns={
    'temp': 'origin_temp',
    'wind_speed': 'origin_wind_speed',
    'description': 'origin_description',
    'humidity': 'origin_humidity',
    'pressure': 'origin_pressure',
    'visibility': 'origin_visibility',
    'precip': 'origin_precip'
}, inplace=True)

In [81]:
origin_weather_merge = origin_weather_merge.drop(columns='datetime')
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,origin_temp,origin_wind_speed,origin_description,origin_precip,origin_humidity,origin_visibility,origin_pressure
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,34.380001,C-c92599e2,68.0,4.0,Light rain shower,0.0,92.0,6.0,1019.0
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,7.620000,C-280b55fb,23.0,4.0,Cloudy,0.0,92.0,6.0,1032.0
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,4.600000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,9.870000,C-ef47bdcd,16.0,9.0,Sunny,0.0,67.0,6.0,1029.0
5,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,33.369999,C-ef47bdcd,28.0,7.0,Heavy snow,0.0,85.0,1.0,1017.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,20.809999,C-419cd14c,54.0,5.0,Partly cloudy,0.0,89.0,6.0,1014.0
11445,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,20.750000,C-419cd14c,63.0,6.0,Overcast,0.0,81.0,6.0,1015.0
11446,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,15.810000,C-e5bfb4e5,-4.0,6.0,Overcast,0.0,84.0,6.0,1025.0
11447,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,2.150000,C-6df8beaf,28.0,6.0,Moderate snow,0.0,93.0,2.0,1028.0


In [82]:
origin_weather_merge.isnull().any()

truck_id                          False
route_id                          False
departure_date                    False
estimated_arrival                 False
delay                             False
estimated_arrival_nearest_hour    False
departure_date_nearest_hour       False
origin_id                         False
destination_id                    False
distance                          False
average_hours                     False
city_id                           False
origin_temp                       False
origin_wind_speed                 False
origin_description                False
origin_precip                     False
origin_humidity                   False
origin_visibility                 False
origin_pressure                   False
dtype: bool

In [83]:
origin_weather_merge = origin_weather_merge.drop_duplicates()

In [84]:
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,origin_temp,origin_wind_speed,origin_description,origin_precip,origin_humidity,origin_visibility,origin_pressure
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,34.380001,C-c92599e2,68.0,4.0,Light rain shower,0.0,92.0,6.0,1019.0
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,7.620000,C-280b55fb,23.0,4.0,Cloudy,0.0,92.0,6.0,1032.0
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,4.600000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,9.870000,C-ef47bdcd,16.0,9.0,Sunny,0.0,67.0,6.0,1029.0
5,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,33.369999,C-ef47bdcd,28.0,7.0,Heavy snow,0.0,85.0,1.0,1017.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,20.809999,C-419cd14c,54.0,5.0,Partly cloudy,0.0,89.0,6.0,1014.0
11445,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,20.750000,C-419cd14c,63.0,6.0,Overcast,0.0,81.0,6.0,1015.0
11446,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,15.810000,C-e5bfb4e5,-4.0,6.0,Overcast,0.0,84.0,6.0,1025.0
11447,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,2.150000,C-6df8beaf,28.0,6.0,Moderate snow,0.0,93.0,2.0,1028.0


In [85]:
origin_weather_merge = origin_weather_merge.drop_duplicates()
origin_weather_merge.shape

(10897, 19)

In [86]:
destination_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-40a81eb1,39,16,Heavy rain,0.2,99,3,1001,2019-01-20 06:00:00
1,C-6df8beaf,28,4,Heavy snow,0.0,95,1,1024,2019-01-12 07:00:00
2,C-639c5e36,27,11,Sunny,0.0,78,6,1022,2019-02-12 16:00:00
3,C-e5bfb4e5,18,6,Light snow,0.0,96,6,1027,2019-02-11 02:00:00
4,C-731988ba,84,6,Sunny,0.0,49,6,1011,2019-02-13 13:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-2aaf0e1a,27,9,Sunny,0.0,31,6,1041,2019-02-09 13:00:00
51830,C-b5282c3b,34,10,Clear,0.0,92,6,1008,2019-01-05 05:00:00
51831,C-c4565ee8,27,4,Cloudy,0.0,92,6,1027,2019-01-17 00:00:00
51832,C-f5ed4c15,18,2,Partly cloudy,0.0,54,6,1025,2019-02-09 19:00:00


In [87]:
final_merge = pd.merge(origin_weather_merge, destination_weather, left_on=['destination_id', 'estimated_arrival_nearest_hour'],
                       right_on=['city_id', 'datetime'], how='left', suffixes=('_origin', '_destination'))

In [88]:
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_pressure,city_id_destination,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,...,1019.0,C-5d86b887,32.0,4.0,Heavy snow,0.0,81.0,2.0,1033.0,2019-01-12 17:00:00
1,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,...,1032.0,C-9bbf5d8a,32.0,9.0,Overcast,0.0,91.0,6.0,1022.0,2019-01-13 15:00:00
2,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,...,1010.0,C-e5bfb4e5,34.0,8.0,Mist,0.0,95.0,1.0,1008.0,2019-01-04 12:00:00
3,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,...,1029.0,C-e6dcda92,25.0,11.0,Patchy light snow,0.0,82.0,6.0,1022.0,2019-01-25 17:00:00
4,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,...,1017.0,C-56e39a5e,81.0,6.0,Sunny,0.0,59.0,6.0,1014.0,2019-01-12 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,...,1014.0,C-280b55fb,0.0,12.0,Clear,0.0,88.0,6.0,1022.0,2019-01-29 04:00:00
10893,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,...,1015.0,C-9bbf5d8a,32.0,2.0,Cloudy,0.0,43.0,6.0,1033.0,2019-01-23 04:00:00
10894,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,...,1025.0,C-9bbf5d8a,34.0,3.0,Clear,0.0,49.0,6.0,1025.0,2019-01-25 23:00:00
10895,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,...,1028.0,C-fc66f0ab,25.0,4.0,Partly cloudy,0.0,91.0,6.0,1030.0,2019-01-13 09:00:00


In [89]:
final_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
       'origin_id', 'destination_id', 'distance', 'average_hours',
       'city_id_origin', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'city_id_destination', 'temp',
       'wind_speed', 'description', 'precip', 'humidity', 'visibility',
       'pressure', 'datetime'],
      dtype='object')

In [90]:
final_merge = final_merge.dropna()
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_pressure,city_id_destination,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,...,1019.0,C-5d86b887,32.0,4.0,Heavy snow,0.0,81.0,2.0,1033.0,2019-01-12 17:00:00
1,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,...,1032.0,C-9bbf5d8a,32.0,9.0,Overcast,0.0,91.0,6.0,1022.0,2019-01-13 15:00:00
2,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,...,1010.0,C-e5bfb4e5,34.0,8.0,Mist,0.0,95.0,1.0,1008.0,2019-01-04 12:00:00
3,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,...,1029.0,C-e6dcda92,25.0,11.0,Patchy light snow,0.0,82.0,6.0,1022.0,2019-01-25 17:00:00
4,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,...,1017.0,C-56e39a5e,81.0,6.0,Sunny,0.0,59.0,6.0,1014.0,2019-01-12 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,...,1014.0,C-280b55fb,0.0,12.0,Clear,0.0,88.0,6.0,1022.0,2019-01-29 04:00:00
10893,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,...,1015.0,C-9bbf5d8a,32.0,2.0,Cloudy,0.0,43.0,6.0,1033.0,2019-01-23 04:00:00
10894,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,...,1025.0,C-9bbf5d8a,34.0,3.0,Clear,0.0,49.0,6.0,1025.0,2019-01-25 23:00:00
10895,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,...,1028.0,C-fc66f0ab,25.0,4.0,Partly cloudy,0.0,91.0,6.0,1030.0,2019-01-13 09:00:00


In [91]:
final_merge.rename(columns={
    'temp': 'dest_temp',
    'wind_speed': 'dest_wind_speed',
    'description': 'dest_description',
    'humidity': 'dest_humidity',
    'pressure': 'dest_pressure',
    'precip': 'dest_precip',
    'visibility': 'dest_visibility'
}, inplace=True)

2024-10-01 16:38:32,157 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [92]:
final_merge = final_merge.drop(
    columns=['datetime', 'city_id_origin', 'city_id_destination'])
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_humidity,origin_visibility,origin_pressure,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,...,92.0,6.0,1019.0,32.0,4.0,Heavy snow,0.0,81.0,2.0,1033.0
1,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,...,92.0,6.0,1032.0,32.0,9.0,Overcast,0.0,91.0,6.0,1022.0
2,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,...,81.0,6.0,1010.0,34.0,8.0,Mist,0.0,95.0,1.0,1008.0
3,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,...,67.0,6.0,1029.0,25.0,11.0,Patchy light snow,0.0,82.0,6.0,1022.0
4,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,...,85.0,1.0,1017.0,81.0,6.0,Sunny,0.0,59.0,6.0,1014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,...,89.0,6.0,1014.0,0.0,12.0,Clear,0.0,88.0,6.0,1022.0
10893,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,...,81.0,6.0,1015.0,32.0,2.0,Cloudy,0.0,43.0,6.0,1033.0
10894,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,...,84.0,6.0,1025.0,34.0,3.0,Clear,0.0,49.0,6.0,1025.0
10895,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,...,93.0,2.0,1028.0,25.0,4.0,Partly cloudy,0.0,91.0,6.0,1030.0


In [93]:
final_merge = final_merge.drop_duplicates()
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_humidity,origin_visibility,origin_pressure,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,...,92.0,6.0,1019.0,32.0,4.0,Heavy snow,0.0,81.0,2.0,1033.0
1,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,...,92.0,6.0,1032.0,32.0,9.0,Overcast,0.0,91.0,6.0,1022.0
2,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,...,81.0,6.0,1010.0,34.0,8.0,Mist,0.0,95.0,1.0,1008.0
3,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,...,67.0,6.0,1029.0,25.0,11.0,Patchy light snow,0.0,82.0,6.0,1022.0
4,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,...,85.0,1.0,1017.0,81.0,6.0,Sunny,0.0,59.0,6.0,1014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,...,89.0,6.0,1014.0,0.0,12.0,Clear,0.0,88.0,6.0,1022.0
10893,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,...,81.0,6.0,1015.0,32.0,2.0,Cloudy,0.0,43.0,6.0,1033.0
10894,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,...,84.0,6.0,1025.0,34.0,3.0,Clear,0.0,49.0,6.0,1025.0
10895,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,...,93.0,2.0,1028.0,25.0,4.0,Partly cloudy,0.0,91.0,6.0,1030.0


In [94]:
final_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
       'origin_id', 'destination_id', 'distance', 'average_hours',
       'origin_temp', 'origin_wind_speed', 'origin_description',
       'origin_precip', 'origin_humidity', 'origin_visibility',
       'origin_pressure', 'dest_temp', 'dest_wind_speed', 'dest_description',
       'dest_precip', 'dest_humidity', 'dest_visibility', 'dest_pressure'],
      dtype='object')

In [95]:
final_merge.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
origin_temp                       0
origin_wind_speed                 0
origin_description                0
origin_precip                     0
origin_humidity                   0
origin_visibility                 0
origin_pressure                   0
dest_temp                         0
dest_wind_speed                   0
dest_description                  0
dest_precip                       0
dest_humidity                     0
dest_visibility                   0
dest_pressure                     0
dtype: int64

In [96]:
schedule_data = df_truck_schedule.copy()

In [97]:
schedule_data

,truck_id,route_id,departure_date,estimated_arrival,delay
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1
...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0


In [98]:
schedule_data['departure_date'] = pd.to_datetime(
    schedule_data['departure_date'])
schedule_data['estimated_arrival'] = pd.to_datetime(
    schedule_data['estimated_arrival'])

# Round to nearest hour
schedule_data['departure_date'] = schedule_data['departure_date'].dt.round('H')
schedule_data['estimated_arrival'] = schedule_data['estimated_arrival'].dt.round(
    'H')

# Display the updated DataFrame
schedule_data

,truck_id,route_id,departure_date,estimated_arrival,delay
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:00:00,1
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 21:00:00,0
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 15:00:00,0
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 12:00:00,0
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 17:00:00,1
...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 04:00:00,0
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 04:00:00,0
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 23:00:00,0
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:00:00,0


In [99]:
hourly_exploded_scheduled_df = (nearest_hour_schedule_df.assign(custom_date=[pd.date_range(start, end, freq='H')  # Create custom date ranges
                                                                             for start, end
                                                                             # Using departure and estimated arrival times
                                                                             in zip(nearest_hour_schedule_df['departure_date'], nearest_hour_schedule_df['estimated_arrival'])])
                                # Explode the DataFrame based on the custom date range
                                .explode('custom_date', ignore_index=True))

In [100]:
df_traffic['datetime'] = pd.to_datetime(df_traffic['datetime'])

In [101]:
df_traffic

,route_id,no_of_vehicles,accident,datetime
0,R-427d68b1,2226.0,0,2019-02-04 17:00:00
1,R-35dca729,556.0,0,2019-01-24 05:00:00
2,R-98b76ad5,2226.0,0,2019-02-11 23:00:00
3,R-1500f777,556.0,0,2019-01-11 02:00:00
4,R-2beaa71d,688.0,0,2019-02-13 04:00:00
...,...,...,...,...
2459677,R-0ecac6ac,2551.0,0,2019-02-15 13:00:00
2459678,R-91e5f4ae,2403.0,0,2019-01-27 08:00:00
2459679,R-8fdff50b,163.0,0,2019-02-01 07:00:00
2459680,R-b5a4476c,2106.0,0,2019-01-02 14:00:00


In [102]:
# df_traffic['end_time'] = df_traffic['datetime'] + pd.Timedelta(hours=24)

In [103]:
# df_traffic = (df_traffic.assign(custom_date=[
#     pd.date_range(start, end, freq='6H')  # Create hourly date ranges
#     for start, end in zip(df_traffic['datetime'], df_traffic['end_time'])
# ]).explode('custom_date', ignore_index=True))

In [104]:
hourly_exploded_scheduled_df.head()

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 07:00:00
1,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 08:00:00
2,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 09:00:00
3,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 10:00:00
4,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 11:00:00


In [105]:
hourly_exploded_scheduled_df = hourly_exploded_scheduled_df.drop_duplicates()

In [106]:
hourly_exploded_scheduled_df

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 07:00:00
1,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 08:00:00
2,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 09:00:00
3,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 10:00:00
4,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 11:00:00
...,...,...,...,...,...,...,...,...
311568,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 16:00:00
311569,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 17:00:00
311570,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 18:00:00
311571,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 19:00:00


In [107]:
df_traffic

,route_id,no_of_vehicles,accident,datetime
0,R-427d68b1,2226.0,0,2019-02-04 17:00:00
1,R-35dca729,556.0,0,2019-01-24 05:00:00
2,R-98b76ad5,2226.0,0,2019-02-11 23:00:00
3,R-1500f777,556.0,0,2019-01-11 02:00:00
4,R-2beaa71d,688.0,0,2019-02-13 04:00:00
...,...,...,...,...
2459677,R-0ecac6ac,2551.0,0,2019-02-15 13:00:00
2459678,R-91e5f4ae,2403.0,0,2019-01-27 08:00:00
2459679,R-8fdff50b,163.0,0,2019-02-01 07:00:00
2459680,R-b5a4476c,2106.0,0,2019-01-02 14:00:00


In [108]:
scheduled_traffic = hourly_exploded_scheduled_df.merge(df_traffic, left_on=[
                                                       'route_id', 'custom_date'], right_on=['route_id', 'datetime'], how='left')

In [109]:
scheduled_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date,no_of_vehicles,accident,datetime
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 07:00:00,1741.0,0.0,2019-01-11 07:00:00
1,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 08:00:00,2758.0,0.0,2019-01-11 08:00:00
2,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 09:00:00,2475.0,0.0,2019-01-11 09:00:00
3,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 10:00:00,1873.0,0.0,2019-01-11 10:00:00
4,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 11:00:00,2512.0,0.0,2019-01-11 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...
311660,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 16:00:00,1875.0,0.0,2019-01-16 16:00:00
311661,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 17:00:00,2596.0,0.0,2019-01-16 17:00:00
311662,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 18:00:00,2403.0,0.0,2019-01-16 18:00:00
311663,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 19:00:00,2532.0,0.0,2019-01-16 19:00:00


In [110]:
scheduled_traffic.isnull().sum()

truck_id                              0
route_id                              0
departure_date                        0
estimated_arrival                     0
delay                                 0
estimated_arrival_nearest_hour        0
departure_date_nearest_hour           0
custom_date                           0
no_of_vehicles                    16317
accident                          16317
datetime                          16317
dtype: int64

In [111]:
scheduled_traffic = scheduled_traffic.dropna()
scheduled_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date,no_of_vehicles,accident,datetime
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 07:00:00,1741.0,0.0,2019-01-11 07:00:00
1,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 08:00:00,2758.0,0.0,2019-01-11 08:00:00
2,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 09:00:00,2475.0,0.0,2019-01-11 09:00:00
3,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 10:00:00,1873.0,0.0,2019-01-11 10:00:00
4,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 11:00:00,2512.0,0.0,2019-01-11 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...
311660,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 16:00:00,1875.0,0.0,2019-01-16 16:00:00
311661,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 17:00:00,2596.0,0.0,2019-01-16 17:00:00
311662,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 18:00:00,2403.0,0.0,2019-01-16 18:00:00
311663,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 19:00:00,2532.0,0.0,2019-01-16 19:00:00


In [112]:
scheduled_traffic = scheduled_traffic.drop_duplicates()
scheduled_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date,no_of_vehicles,accident,datetime
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 07:00:00,1741.0,0.0,2019-01-11 07:00:00
1,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 08:00:00,2758.0,0.0,2019-01-11 08:00:00
2,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 09:00:00,2475.0,0.0,2019-01-11 09:00:00
3,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 10:00:00,1873.0,0.0,2019-01-11 10:00:00
4,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,2019-01-11 11:00:00,2512.0,0.0,2019-01-11 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...
311660,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 16:00:00,1875.0,0.0,2019-01-16 16:00:00
311661,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 17:00:00,2596.0,0.0,2019-01-16 17:00:00
311662,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 18:00:00,2403.0,0.0,2019-01-16 18:00:00
311663,14730346,R-dda44c57,2019-01-16 07:00:00,2019-01-16 20:54:00,0,2019-01-16 21:00:00,2019-01-16 07:00:00,2019-01-16 19:00:00,2532.0,0.0,2019-01-16 19:00:00


In [113]:
scheduled_traffic.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
custom_date                       0
no_of_vehicles                    0
accident                          0
datetime                          0
dtype: int64

In [114]:
def custom_agg(values):
    if any(values == 1):
        return 1
    else:
        return 0

In [115]:
scheduled_route_traffic = scheduled_traffic.groupby(['truck_id', 'route_id'], as_index=False).agg(
    avg_no_of_vehicles=('no_of_vehicles', 'mean'),
    accident=('accident', custom_agg)
)

In [116]:
scheduled_route_traffic

,truck_id,route_id,avg_no_of_vehicles,accident
0,10008392,R-06dfe39e,1883.086182,0
1,10008392,R-35dca729,1833.734741,0
2,10008392,R-51d1a32a,1522.763672,0
3,10008392,R-7ab5ae85,1976.000000,0
4,10008392,R-c6f8e036,812.438599,0
...,...,...,...,...
12227,99981667,R-abbf83d1,1792.812500,0
12228,99981667,R-e9bf94ff,2074.166748,0
12229,99981667,R-ed1094fd,1915.666626,0
12230,99981667,R-f363ad37,1831.750000,0


In [117]:
scheduled_route_traffic.isnull().sum()

truck_id              0
route_id              0
avg_no_of_vehicles    0
accident              0
dtype: int64

In [118]:
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_humidity,origin_visibility,origin_pressure,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,...,92.0,6.0,1019.0,32.0,4.0,Heavy snow,0.0,81.0,2.0,1033.0
1,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,...,92.0,6.0,1032.0,32.0,9.0,Overcast,0.0,91.0,6.0,1022.0
2,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,...,81.0,6.0,1010.0,34.0,8.0,Mist,0.0,95.0,1.0,1008.0
3,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,...,67.0,6.0,1029.0,25.0,11.0,Patchy light snow,0.0,82.0,6.0,1022.0
4,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,...,85.0,1.0,1017.0,81.0,6.0,Sunny,0.0,59.0,6.0,1014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,...,89.0,6.0,1014.0,0.0,12.0,Clear,0.0,88.0,6.0,1022.0
10893,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,...,81.0,6.0,1015.0,32.0,2.0,Cloudy,0.0,43.0,6.0,1033.0
10894,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,...,84.0,6.0,1025.0,34.0,3.0,Clear,0.0,49.0,6.0,1025.0
10895,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,...,93.0,2.0,1028.0,25.0,4.0,Partly cloudy,0.0,91.0,6.0,1030.0


In [119]:
final_merge.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
origin_temp                       0
origin_wind_speed                 0
origin_description                0
origin_precip                     0
origin_humidity                   0
origin_visibility                 0
origin_pressure                   0
dest_temp                         0
dest_wind_speed                   0
dest_description                  0
dest_precip                       0
dest_humidity                     0
dest_visibility                   0
dest_pressure                     0
dtype: int64

In [120]:
scheduled_route_traffic

,truck_id,route_id,avg_no_of_vehicles,accident
0,10008392,R-06dfe39e,1883.086182,0
1,10008392,R-35dca729,1833.734741,0
2,10008392,R-51d1a32a,1522.763672,0
3,10008392,R-7ab5ae85,1976.000000,0
4,10008392,R-c6f8e036,812.438599,0
...,...,...,...,...
12227,99981667,R-abbf83d1,1792.812500,0
12228,99981667,R-e9bf94ff,2074.166748,0
12229,99981667,R-ed1094fd,1915.666626,0
12230,99981667,R-f363ad37,1831.750000,0


In [121]:
origin_destination_weather_traffic_merge = final_merge.merge(
    scheduled_route_traffic, on=['truck_id', 'route_id'], how='left')

In [122]:
origin_destination_weather_traffic_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_pressure,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,avg_no_of_vehicles,accident
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,...,1019.0,32.0,4.0,Heavy snow,0.0,81.0,2.0,1033.0,1944.687500,0.0
1,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,...,1032.0,32.0,9.0,Overcast,0.0,91.0,6.0,1022.0,2269.125000,0.0
2,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,...,1010.0,34.0,8.0,Mist,0.0,95.0,1.0,1008.0,2194.199951,0.0
3,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,...,1029.0,25.0,11.0,Patchy light snow,0.0,82.0,6.0,1022.0,2129.500000,0.0
4,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,...,1017.0,81.0,6.0,Sunny,0.0,59.0,6.0,1014.0,258.812500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10701,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,...,1014.0,0.0,12.0,Clear,0.0,88.0,6.0,1022.0,1938.952393,0.0
10702,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,...,1015.0,32.0,2.0,Cloudy,0.0,43.0,6.0,1033.0,2073.333252,0.0
10703,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,...,1025.0,34.0,3.0,Clear,0.0,49.0,6.0,1025.0,2235.266602,0.0
10704,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,...,1028.0,25.0,4.0,Partly cloudy,0.0,91.0,6.0,1030.0,503.000000,0.0


In [123]:
origin_destination_weather_traffic_merge.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
origin_temp                       0
origin_wind_speed                 0
origin_description                0
origin_precip                     0
origin_humidity                   0
origin_visibility                 0
origin_pressure                   0
dest_temp                         0
dest_wind_speed                   0
dest_description                  0
dest_precip                       0
dest_humidity                     0
dest_visibility                   0
dest_pressure                     0
avg_no_of_vehicles                7
accident                          7
dtype: int64

In [124]:
origin_destination_weather_traffic_merge = origin_destination_weather_traffic_merge.dropna()

In [125]:
# origin_destination_weather_traffic_merge = origin_destination_weather_traffic_merge.drop(columns='index_y')

In [126]:
origin_destination_weather_traffic_merge = origin_destination_weather_traffic_merge.drop_duplicates()
origin_destination_weather_traffic_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_pressure,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,avg_no_of_vehicles,accident
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,...,1019.0,32.0,4.0,Heavy snow,0.0,81.0,2.0,1033.0,1944.687500,0.0
1,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,...,1032.0,32.0,9.0,Overcast,0.0,91.0,6.0,1022.0,2269.125000,0.0
2,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,...,1010.0,34.0,8.0,Mist,0.0,95.0,1.0,1008.0,2194.199951,0.0
3,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,...,1029.0,25.0,11.0,Patchy light snow,0.0,82.0,6.0,1022.0,2129.500000,0.0
4,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,...,1017.0,81.0,6.0,Sunny,0.0,59.0,6.0,1014.0,258.812500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10701,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,...,1014.0,0.0,12.0,Clear,0.0,88.0,6.0,1022.0,1938.952393,0.0
10702,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,...,1015.0,32.0,2.0,Cloudy,0.0,43.0,6.0,1033.0,2073.333252,0.0
10703,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,...,1025.0,34.0,3.0,Clear,0.0,49.0,6.0,1025.0,2235.266602,0.0
10704,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,...,1028.0,25.0,4.0,Partly cloudy,0.0,91.0,6.0,1030.0,503.000000,0.0


In [127]:
origin_destination_weather_traffic_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
       'origin_id', 'destination_id', 'distance', 'average_hours',
       'origin_temp', 'origin_wind_speed', 'origin_description',
       'origin_precip', 'origin_humidity', 'origin_visibility',
       'origin_pressure', 'dest_temp', 'dest_wind_speed', 'dest_description',
       'dest_precip', 'dest_humidity', 'dest_visibility', 'dest_pressure',
       'avg_no_of_vehicles', 'accident'],
      dtype='object')

In [128]:
origin_destination_weather_traffic_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_pressure,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,avg_no_of_vehicles,accident
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,...,1019.0,32.0,4.0,Heavy snow,0.0,81.0,2.0,1033.0,1944.687500,0.0
1,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,...,1032.0,32.0,9.0,Overcast,0.0,91.0,6.0,1022.0,2269.125000,0.0
2,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,...,1010.0,34.0,8.0,Mist,0.0,95.0,1.0,1008.0,2194.199951,0.0
3,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,...,1029.0,25.0,11.0,Patchy light snow,0.0,82.0,6.0,1022.0,2129.500000,0.0
4,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,...,1017.0,81.0,6.0,Sunny,0.0,59.0,6.0,1014.0,258.812500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10701,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,...,1014.0,0.0,12.0,Clear,0.0,88.0,6.0,1022.0,1938.952393,0.0
10702,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,...,1015.0,32.0,2.0,Cloudy,0.0,43.0,6.0,1033.0,2073.333252,0.0
10703,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,...,1025.0,34.0,3.0,Clear,0.0,49.0,6.0,1025.0,2235.266602,0.0
10704,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,...,1028.0,25.0,4.0,Partly cloudy,0.0,91.0,6.0,1030.0,503.000000,0.0


In [129]:
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,61.285714,4.000000,62.142857,1013.285714,0.0,4.285714,Sunny
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0,80.500000,10.000000,63.500000,1011.250000,0.0,4.250000,Patchy light drizzle
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.000000,0.0,6.000000,Clear
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.500000,0.0,6.000000,Clear
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.000000,0.0,4.333333,Moderate snow
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.200000,0.0,6.000000,Sunny
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.000000,0.0,5.000000,Sunny
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,63.000000,6.000000,27.500000,1021.000000,0.0,6.000000,Clear
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,60.000000,8.500000,65.000000,1018.500000,0.0,6.000000,Cloudy


In [130]:
schedule_weather_merge.isnull().sum()

truck_id                0
route_id                0
departure_date          0
estimated_arrival       0
delay                   0
route_avg_temp          0
route_avg_wind_speed    0
route_avg_humidity      0
route_avg_pressure      0
route_avg_precip        0
route_avg_visibility    0
route_description       0
dtype: int64

In [131]:
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,61.285714,4.000000,62.142857,1013.285714,0.0,4.285714,Sunny
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0,80.500000,10.000000,63.500000,1011.250000,0.0,4.250000,Patchy light drizzle
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.000000,0.0,6.000000,Clear
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.500000,0.0,6.000000,Clear
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.000000,0.0,4.333333,Moderate snow
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.200000,0.0,6.000000,Sunny
12304,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.000000,0.0,5.000000,Sunny
12305,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,63.000000,6.000000,27.500000,1021.000000,0.0,6.000000,Clear
12306,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,60.000000,8.500000,65.000000,1018.500000,0.0,6.000000,Cloudy


In [132]:
merged_data_weather_traffic = pd.merge(schedule_weather_merge, origin_destination_weather_traffic_merge, on=[
                                       'truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay'], how='left')

In [133]:
origin_destination_weather_traffic_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_pressure,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,avg_no_of_vehicles,accident
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,2019-01-12 17:00:00,2019-01-11 07:00:00,C-c92599e2,C-5d86b887,1719.189941,...,1019.0,32.0,4.0,Heavy snow,0.0,81.0,2.0,1033.0,1944.687500,0.0
1,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,2019-01-13 15:00:00,2019-01-13 07:00:00,C-280b55fb,C-9bbf5d8a,380.839996,...,1032.0,32.0,9.0,Overcast,0.0,91.0,6.0,1022.0,2269.125000,0.0
2,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,2019-01-04 12:00:00,2019-01-04 07:00:00,C-b5282c3b,C-e5bfb4e5,229.759995,...,1010.0,34.0,8.0,Mist,0.0,95.0,1.0,1008.0,2194.199951,0.0
3,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,2019-01-25 17:00:00,2019-01-25 07:00:00,C-ef47bdcd,C-e6dcda92,493.440002,...,1029.0,25.0,11.0,Patchy light snow,0.0,82.0,6.0,1022.0,2129.500000,0.0
4,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,2019-01-12 16:00:00,2019-01-11 07:00:00,C-ef47bdcd,C-56e39a5e,1668.640015,...,1017.0,81.0,6.0,Sunny,0.0,59.0,6.0,1014.0,258.812500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10701,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,2019-01-29 04:00:00,2019-01-28 07:00:00,C-419cd14c,C-280b55fb,1040.660034,...,1014.0,0.0,12.0,Clear,0.0,88.0,6.0,1022.0,1938.952393,0.0
10702,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,2019-01-23 04:00:00,2019-01-22 07:00:00,C-419cd14c,C-9bbf5d8a,1037.739990,...,1015.0,32.0,2.0,Cloudy,0.0,43.0,6.0,1033.0,2073.333252,0.0
10703,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,2019-01-25 23:00:00,2019-01-25 07:00:00,C-e5bfb4e5,C-9bbf5d8a,790.340027,...,1025.0,34.0,3.0,Clear,0.0,49.0,6.0,1025.0,2235.266602,0.0
10704,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,2019-01-13 09:00:00,2019-01-13 07:00:00,C-6df8beaf,C-fc66f0ab,107.500000,...,1028.0,25.0,4.0,Partly cloudy,0.0,91.0,6.0,1030.0,503.000000,0.0


In [134]:
origin_destination_weather_traffic_merge.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
origin_temp                       0
origin_wind_speed                 0
origin_description                0
origin_precip                     0
origin_humidity                   0
origin_visibility                 0
origin_pressure                   0
dest_temp                         0
dest_wind_speed                   0
dest_description                  0
dest_precip                       0
dest_humidity                     0
dest_visibility                   0
dest_pressure                     0
avg_no_of_vehicles                0
accident                          0
dtype: int64

In [135]:
merged_data_weather_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,origin_pressure,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,avg_no_of_vehicles,accident
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,61.285714,4.000000,62.142857,1013.285714,0.0,...,1019.0,32.0,4.0,Heavy snow,0.0,81.0,2.0,1033.0,1944.687500,0.0
1,61984883,R-d87e53cd,2019-02-09 07:00:00,2019-02-09 20:38:24,0,80.500000,10.000000,63.500000,1011.250000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.000000,0.0,...,1032.0,32.0,9.0,Overcast,0.0,91.0,6.0,1022.0,2269.125000,0.0
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.500000,0.0,...,1010.0,34.0,8.0,Mist,0.0,95.0,1.0,1008.0,2194.199951,0.0
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.000000,0.0,...,1029.0,25.0,11.0,Patchy light snow,0.0,82.0,6.0,1022.0,2129.500000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12260,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.200000,0.0,...,1014.0,0.0,12.0,Clear,0.0,88.0,6.0,1022.0,1938.952393,0.0
12261,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.000000,0.0,...,1015.0,32.0,2.0,Cloudy,0.0,43.0,6.0,1033.0,2073.333252,0.0
12262,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,63.000000,6.000000,27.500000,1021.000000,0.0,...,1025.0,34.0,3.0,Clear,0.0,49.0,6.0,1025.0,2235.266602,0.0
12263,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,60.000000,8.500000,65.000000,1018.500000,0.0,...,1028.0,25.0,4.0,Partly cloudy,0.0,91.0,6.0,1030.0,503.000000,0.0


In [136]:
merged_data_weather_traffic.isnull().sum()

truck_id                             0
route_id                             0
departure_date                       0
estimated_arrival                    0
delay                                0
route_avg_temp                       0
route_avg_wind_speed                 0
route_avg_humidity                   0
route_avg_pressure                   0
route_avg_precip                     0
route_avg_visibility                 0
route_description                    0
estimated_arrival_nearest_hour    1607
departure_date_nearest_hour       1607
origin_id                         1607
destination_id                    1607
distance                          1607
average_hours                     1607
origin_temp                       1607
origin_wind_speed                 1607
origin_description                1607
origin_precip                     1607
origin_humidity                   1607
origin_visibility                 1607
origin_pressure                   1607
dest_temp                

In [137]:
merged_data_weather_traffic = merged_data_weather_traffic.dropna()

In [138]:
merged_data_weather_traffic = merged_data_weather_traffic.drop_duplicates()

In [139]:
merged_data_weather_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,origin_pressure,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,avg_no_of_vehicles,accident
0,26716985,R-d348d1e2,2019-01-11 07:00:00,2019-01-12 17:22:48,1,61.285714,4.000000,62.142857,1013.285714,0.0,...,1019.0,32.0,4.0,Heavy snow,0.0,81.0,2.0,1033.0,1944.687500,0.0
2,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.000000,0.0,...,1032.0,32.0,9.0,Overcast,0.0,91.0,6.0,1022.0,2269.125000,0.0
3,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.500000,0.0,...,1010.0,34.0,8.0,Mist,0.0,95.0,1.0,1008.0,2194.199951,0.0
4,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.000000,0.0,...,1029.0,25.0,11.0,Patchy light snow,0.0,82.0,6.0,1022.0,2129.500000,0.0
5,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,70.500000,2.000000,85.250000,1014.250000,0.0,...,1017.0,81.0,6.0,Sunny,0.0,59.0,6.0,1014.0,258.812500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12260,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.200000,0.0,...,1014.0,0.0,12.0,Clear,0.0,88.0,6.0,1022.0,1938.952393,0.0
12261,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.000000,0.0,...,1015.0,32.0,2.0,Cloudy,0.0,43.0,6.0,1033.0,2073.333252,0.0
12262,24034677,R-4febf0c1,2019-01-25 07:00:00,2019-01-25 22:48:36,0,63.000000,6.000000,27.500000,1021.000000,0.0,...,1025.0,34.0,3.0,Clear,0.0,49.0,6.0,1025.0,2235.266602,0.0
12263,33921946,R-24c815d8,2019-01-13 07:00:00,2019-01-13 09:09:00,0,60.000000,8.500000,65.000000,1018.500000,0.0,...,1028.0,25.0,4.0,Partly cloudy,0.0,91.0,6.0,1030.0,503.000000,0.0


In [140]:
merged_data_weather_traffic_trucks = pd.merge(
    merged_data_weather_traffic, df_trucks, on='truck_id', how='left')

In [141]:
df_trucks

,truck_id,truck_age,load_capacity_pounds,mileage_mpg,fuel_type
0,18580086,7,6000.0,24,diesel
1,26225733,10,3000.0,21,diesel
2,22377469,12,15000.0,16,gas
3,96814728,12,3000.0,23,diesel
4,30794834,7,10000.0,18,gas
...,...,...,...,...,...
1196,21865328,10,3000.0,15,gas
1197,27274345,10,15000.0,23,diesel
1198,24624497,11,4000.0,18,gas
1199,12323896,11,15000.0,19,gas


In [142]:
merged_data_weather_traffic_trucks.isnull().sum()

truck_id                            0
route_id                            0
departure_date                      0
estimated_arrival                   0
delay                               0
route_avg_temp                      0
route_avg_wind_speed                0
route_avg_humidity                  0
route_avg_pressure                  0
route_avg_precip                    0
route_avg_visibility                0
route_description                   0
estimated_arrival_nearest_hour      0
departure_date_nearest_hour         0
origin_id                           0
destination_id                      0
distance                            0
average_hours                       0
origin_temp                         0
origin_wind_speed                   0
origin_description                  0
origin_precip                       0
origin_humidity                     0
origin_visibility                   0
origin_pressure                     0
dest_temp                           0
dest_wind_sp

In [143]:
merged_data_weather_traffic_trucks = merged_data_weather_traffic_trucks.dropna()

In [144]:
merged_data_weather_traffic_trucks

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,dest_precip,dest_humidity,dest_visibility,dest_pressure,avg_no_of_vehicles,accident,truck_age,load_capacity_pounds,mileage_mpg,fuel_type
1,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.00,0.00,...,0.0,91.0,6.0,1022.0,2269.125000,0.0,11.0,6000.0,25.0,diesel
2,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.50,0.00,...,0.0,95.0,1.0,1008.0,2194.199951,0.0,10.0,4000.0,24.0,diesel
3,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.00,0.00,...,0.0,82.0,6.0,1022.0,2129.500000,0.0,14.0,6000.0,17.0,gas
4,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,70.500000,2.000000,85.250000,1014.25,0.00,...,0.0,59.0,6.0,1014.0,258.812500,0.0,10.0,6000.0,25.0,diesel
5,80740040,R-abc9e78b,2019-01-28 07:00:00,2019-01-29 05:28:12,1,50.000000,1.800000,77.800000,1016.00,0.00,...,0.0,49.0,6.0,1028.0,1808.857178,0.0,12.0,15000.0,21.0,diesel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10651,15060086,R-99fd990d,2019-02-06 07:00:00,2019-02-07 01:17:23,1,32.000000,15.000000,97.000000,1001.50,0.05,...,0.0,90.0,2.0,1015.0,2069.277832,0.0,14.0,15000.0,18.0,gas
10652,21001184,R-e45a1037,2019-01-10 07:00:00,2019-01-10 17:37:48,0,74.666667,6.333333,45.333333,1010.00,0.00,...,0.0,70.0,6.0,1029.0,2415.600098,0.0,11.0,15000.0,17.0,gas
10653,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.20,0.00,...,0.0,88.0,6.0,1022.0,1938.952393,0.0,12.0,15000.0,18.0,gas
10654,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.00,0.00,...,0.0,43.0,6.0,1033.0,2073.333252,0.0,9.0,10000.0,13.0,gas


In [145]:
final_merge_dataset = pd.merge(merged_data_weather_traffic_trucks,
                               df_drivers, left_on='truck_id', right_on='vehicle_no', how='left')

In [146]:
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,fuel_type,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph
0,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.00,0.00,...,diesel,9f19a100-6,James Kelley,male,56.0,30.0,proactive,9.0,18663378.0,60.290001
1,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.50,0.00,...,diesel,cff19411-5,Scott Lucas,male,50.0,11.0,proactive,5.0,23496767.0,63.000000
2,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.00,0.00,...,gas,3803b114-1,Brandon Allen,male,48.0,10.0,conservative,2.0,32898016.0,48.169998
3,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,70.500000,2.000000,85.250000,1014.25,0.00,...,diesel,6621c3cd-7,Kristopher Rivera,male,38.0,4.0,proactive,7.0,28188509.0,66.480003
4,80740040,R-abc9e78b,2019-01-28 07:00:00,2019-01-29 05:28:12,1,50.000000,1.800000,77.800000,1016.00,0.00,...,diesel,2a80835e-b,Jonathan Ramirez,male,42.0,10.0,conservative,7.0,80740040.0,47.410000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9768,15060086,R-99fd990d,2019-02-06 07:00:00,2019-02-07 01:17:23,1,32.000000,15.000000,97.000000,1001.50,0.05,...,gas,faacdded-2,Richard Williams,male,55.0,19.0,conservative,7.0,15060086.0,41.220001
9769,21001184,R-e45a1037,2019-01-10 07:00:00,2019-01-10 17:37:48,0,74.666667,6.333333,45.333333,1010.00,0.00,...,gas,0fc4a002-7,Andrew Williams,male,51.0,18.0,proactive,6.0,21001184.0,60.209999
9770,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.20,0.00,...,gas,ed6b6bac-a,Robin Howell,female,48.0,11.0,conservative,4.0,10822881.0,35.070000
9771,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.00,0.00,...,gas,8a0326ff-c,John Miller,male,49.0,7.0,proactive,3.0,10489983.0,61.349998


In [147]:
def has_midnight(start, end):
    return int(start.date() != end.date())

In [148]:
final_merge_dataset['is_midnight'] = final_merge_dataset.apply(
    lambda row: has_midnight(row['departure_date'], row['estimated_arrival']), axis=1)

In [149]:
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.00,0.00,...,9f19a100-6,James Kelley,male,56.0,30.0,proactive,9.0,18663378.0,60.290001,0
1,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.50,0.00,...,cff19411-5,Scott Lucas,male,50.0,11.0,proactive,5.0,23496767.0,63.000000,0
2,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.00,0.00,...,3803b114-1,Brandon Allen,male,48.0,10.0,conservative,2.0,32898016.0,48.169998,0
3,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,70.500000,2.000000,85.250000,1014.25,0.00,...,6621c3cd-7,Kristopher Rivera,male,38.0,4.0,proactive,7.0,28188509.0,66.480003,1
4,80740040,R-abc9e78b,2019-01-28 07:00:00,2019-01-29 05:28:12,1,50.000000,1.800000,77.800000,1016.00,0.00,...,2a80835e-b,Jonathan Ramirez,male,42.0,10.0,conservative,7.0,80740040.0,47.410000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9768,15060086,R-99fd990d,2019-02-06 07:00:00,2019-02-07 01:17:23,1,32.000000,15.000000,97.000000,1001.50,0.05,...,faacdded-2,Richard Williams,male,55.0,19.0,conservative,7.0,15060086.0,41.220001,1
9769,21001184,R-e45a1037,2019-01-10 07:00:00,2019-01-10 17:37:48,0,74.666667,6.333333,45.333333,1010.00,0.00,...,0fc4a002-7,Andrew Williams,male,51.0,18.0,proactive,6.0,21001184.0,60.209999,0
9770,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.20,0.00,...,ed6b6bac-a,Robin Howell,female,48.0,11.0,conservative,4.0,10822881.0,35.070000,1
9771,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.00,0.00,...,8a0326ff-c,John Miller,male,49.0,7.0,proactive,3.0,10489983.0,61.349998,1


In [150]:
final_merge_dataset.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_humidity',
       'route_avg_pressure', 'route_avg_precip', 'route_avg_visibility',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'dest_temp', 'dest_wind_speed',
       'dest_description', 'dest_precip', 'dest_humidity', 'dest_visibility',
       'dest_pressure', 'avg_no_of_vehicles', 'accident', 'truck_age',
       'load_capacity_pounds', 'mileage_mpg', 'fuel_type', 'driver_id', 'name',
       'gender', 'age', 'experience', 'driving_style', 'ratings', 'vehicle_no',
       'average_speed_mph', 'is_midnight'],
      dtype='object')

In [151]:
final_merge_dataset.shape

(9773, 48)

In [152]:
final_merge_dataset = final_merge_dataset.drop_duplicates()
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.00,0.00,...,9f19a100-6,James Kelley,male,56.0,30.0,proactive,9.0,18663378.0,60.290001,0
1,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.50,0.00,...,cff19411-5,Scott Lucas,male,50.0,11.0,proactive,5.0,23496767.0,63.000000,0
2,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.00,0.00,...,3803b114-1,Brandon Allen,male,48.0,10.0,conservative,2.0,32898016.0,48.169998,0
3,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,70.500000,2.000000,85.250000,1014.25,0.00,...,6621c3cd-7,Kristopher Rivera,male,38.0,4.0,proactive,7.0,28188509.0,66.480003,1
4,80740040,R-abc9e78b,2019-01-28 07:00:00,2019-01-29 05:28:12,1,50.000000,1.800000,77.800000,1016.00,0.00,...,2a80835e-b,Jonathan Ramirez,male,42.0,10.0,conservative,7.0,80740040.0,47.410000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9768,15060086,R-99fd990d,2019-02-06 07:00:00,2019-02-07 01:17:23,1,32.000000,15.000000,97.000000,1001.50,0.05,...,faacdded-2,Richard Williams,male,55.0,19.0,conservative,7.0,15060086.0,41.220001,1
9769,21001184,R-e45a1037,2019-01-10 07:00:00,2019-01-10 17:37:48,0,74.666667,6.333333,45.333333,1010.00,0.00,...,0fc4a002-7,Andrew Williams,male,51.0,18.0,proactive,6.0,21001184.0,60.209999,0
9770,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.20,0.00,...,ed6b6bac-a,Robin Howell,female,48.0,11.0,conservative,4.0,10822881.0,35.070000,1
9771,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.00,0.00,...,8a0326ff-c,John Miller,male,49.0,7.0,proactive,3.0,10489983.0,61.349998,1


In [153]:
final_merge_dataset.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_humidity',
       'route_avg_pressure', 'route_avg_precip', 'route_avg_visibility',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'dest_temp', 'dest_wind_speed',
       'dest_description', 'dest_precip', 'dest_humidity', 'dest_visibility',
       'dest_pressure', 'avg_no_of_vehicles', 'accident', 'truck_age',
       'load_capacity_pounds', 'mileage_mpg', 'fuel_type', 'driver_id', 'name',
       'gender', 'age', 'experience', 'driving_style', 'ratings', 'vehicle_no',
       'average_speed_mph', 'is_midnight'],
      dtype='object')

In [154]:
final_merge_dataset.shape

(9773, 48)

In [155]:
final_merge_dataset.isnull().sum()

truck_id                            0
route_id                            0
departure_date                      0
estimated_arrival                   0
delay                               0
route_avg_temp                      0
route_avg_wind_speed                0
route_avg_humidity                  0
route_avg_pressure                  0
route_avg_precip                    0
route_avg_visibility                0
route_description                   0
estimated_arrival_nearest_hour      0
departure_date_nearest_hour         0
origin_id                           0
destination_id                      0
distance                            0
average_hours                       0
origin_temp                         0
origin_wind_speed                   0
origin_description                  0
origin_precip                       0
origin_humidity                     0
origin_visibility                   0
origin_pressure                     0
dest_temp                           0
dest_wind_sp

In [156]:
final_merge_dataset = final_merge_dataset.dropna()
final_merge_dataset = final_merge_dataset.drop_duplicates()
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.00,0.00,...,9f19a100-6,James Kelley,male,56.0,30.0,proactive,9.0,18663378.0,60.290001,0
1,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.50,0.00,...,cff19411-5,Scott Lucas,male,50.0,11.0,proactive,5.0,23496767.0,63.000000,0
2,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.00,0.00,...,3803b114-1,Brandon Allen,male,48.0,10.0,conservative,2.0,32898016.0,48.169998,0
3,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,70.500000,2.000000,85.250000,1014.25,0.00,...,6621c3cd-7,Kristopher Rivera,male,38.0,4.0,proactive,7.0,28188509.0,66.480003,1
4,80740040,R-abc9e78b,2019-01-28 07:00:00,2019-01-29 05:28:12,1,50.000000,1.800000,77.800000,1016.00,0.00,...,2a80835e-b,Jonathan Ramirez,male,42.0,10.0,conservative,7.0,80740040.0,47.410000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9768,15060086,R-99fd990d,2019-02-06 07:00:00,2019-02-07 01:17:23,1,32.000000,15.000000,97.000000,1001.50,0.05,...,faacdded-2,Richard Williams,male,55.0,19.0,conservative,7.0,15060086.0,41.220001,1
9769,21001184,R-e45a1037,2019-01-10 07:00:00,2019-01-10 17:37:48,0,74.666667,6.333333,45.333333,1010.00,0.00,...,0fc4a002-7,Andrew Williams,male,51.0,18.0,proactive,6.0,21001184.0,60.209999,0
9770,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.20,0.00,...,ed6b6bac-a,Robin Howell,female,48.0,11.0,conservative,4.0,10822881.0,35.070000,1
9771,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.00,0.00,...,8a0326ff-c,John Miller,male,49.0,7.0,proactive,3.0,10489983.0,61.349998,1


In [157]:
final_merge_dataset.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_humidity',
       'route_avg_pressure', 'route_avg_precip', 'route_avg_visibility',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'dest_temp', 'dest_wind_speed',
       'dest_description', 'dest_precip', 'dest_humidity', 'dest_visibility',
       'dest_pressure', 'avg_no_of_vehicles', 'accident', 'truck_age',
       'load_capacity_pounds', 'mileage_mpg', 'fuel_type', 'driver_id', 'name',
       'gender', 'age', 'experience', 'driving_style', 'ratings', 'vehicle_no',
       'average_speed_mph', 'is_midnight'],
      dtype='object')

In [158]:
final_merge_dataset['unique_id'] = range(1, len(final_merge_dataset) + 1)

In [159]:
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight,unique_id
0,18663378,R-d5d0786c,2019-01-13 07:00:00,2019-01-13 14:37:12,0,63.333333,10.000000,91.000000,1014.00,0.00,...,James Kelley,male,56.0,30.0,proactive,9.0,18663378.0,60.290001,0,1
1,23496767,R-6717548a,2019-01-04 07:00:00,2019-01-04 11:36:00,0,45.500000,7.500000,53.500000,1025.50,0.00,...,Scott Lucas,male,50.0,11.0,proactive,5.0,23496767.0,63.000000,0,2
2,32898016,R-c9bd5880,2019-01-25 07:00:00,2019-01-25 16:52:12,1,32.333333,10.333333,75.666667,1015.00,0.00,...,Brandon Allen,male,48.0,10.0,conservative,2.0,32898016.0,48.169998,0,3
3,28188509,R-272a3bd5,2019-01-11 07:00:00,2019-01-12 16:22:12,1,70.500000,2.000000,85.250000,1014.25,0.00,...,Kristopher Rivera,male,38.0,4.0,proactive,7.0,28188509.0,66.480003,1,4
4,80740040,R-abc9e78b,2019-01-28 07:00:00,2019-01-29 05:28:12,1,50.000000,1.800000,77.800000,1016.00,0.00,...,Jonathan Ramirez,male,42.0,10.0,conservative,7.0,80740040.0,47.410000,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9768,15060086,R-99fd990d,2019-02-06 07:00:00,2019-02-07 01:17:23,1,32.000000,15.000000,97.000000,1001.50,0.05,...,Richard Williams,male,55.0,19.0,conservative,7.0,15060086.0,41.220001,1,9455
9769,21001184,R-e45a1037,2019-01-10 07:00:00,2019-01-10 17:37:48,0,74.666667,6.333333,45.333333,1010.00,0.00,...,Andrew Williams,male,51.0,18.0,proactive,6.0,21001184.0,60.209999,0,9456
9770,10822881,R-c6a07410,2019-01-28 07:00:00,2019-01-29 03:48:36,0,51.000000,9.600000,68.600000,1019.20,0.00,...,Robin Howell,female,48.0,11.0,conservative,4.0,10822881.0,35.070000,1,9457
9771,10489983,R-7738be4f,2019-01-22 07:00:00,2019-01-23 03:45:00,0,76.000000,10.400000,62.000000,1011.00,0.00,...,John Miller,male,49.0,7.0,proactive,3.0,10489983.0,61.349998,1,9458


In [160]:
final_merge_dataset.to_csv(
    '/Users/rammaruthi/Desktop/Truck/notebooks/final_merge_dataset.csv')

In [161]:
final_merge_dataset.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
route_avg_temp                    0
route_avg_wind_speed              0
route_avg_humidity                0
route_avg_pressure                0
route_avg_precip                  0
route_avg_visibility              0
route_description                 0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
origin_temp                       0
origin_wind_speed                 0
origin_description                0
origin_precip                     0
origin_humidity                   0
origin_visibility                 0
origin_pressure                   0
dest_temp                         0
dest_wind_speed                   0
dest_description            

In [162]:
final_merge_dataset.shape

(9459, 49)